In [1]:
import pandas as pd
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [17]:
gamel = pd.read_csv('game_log.csv')
parkс = pd.read_csv('park_codes.csv')
personс = pd.read_csv('person_codes.csv')
teamc = pd.read_csv('team_codes.csv')
appeart = pd.read_csv('appearance_type.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = '''SELECT
                name,
                type
            FROM sqlite_master
            WHERE type IN ("table", "view")'''
    
    return run_query(q)

In [19]:
with sqlite3.connect('mlb.db') as conn:
    try:
        gamel.to_sql('game_log', conn, index=False)
        parkс.to_sql('park_codes', conn, index=False)
        personс.to_sql('person_codes', conn, index=False)
        teamc.to_sql('team_codes', conn, index=False)
        appeart.to_sql('appearance_type', conn, index_label='appearance_type_id')
    except ValueError as err:
        print('ValueError:', err)  

In [6]:
q = '''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
'''

run_command(q)

q = '''
UPDATE game_log
SET game_id = h_name || date || number_of_game;
'''

run_command(q)

In [7]:
q = '''
CREATE TABLE IF NOT EXISTS person(
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
'''
run_command(q)

q = '''
CREATE TABLE IF NOT EXISTS park(
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
'''
run_command(q)

q = '''
CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
'''
run_command(q)

In [8]:
q = '''
INSERT OR IGNORE INTO person 
    SELECT                                   
       id,                                                           
       first,
       last
    FROM person_codes
'''
run_command(q)   

result = run_query('SELECT * FROM person LIMIT 5')
result

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [9]:
q = '''
INSERT OR IGNORE INTO park 
    SELECT                                   
       park_id,                                                           
       name,
       aka,
       city,
       state,
       notes
    FROM park_codes
'''
run_command(q)   

result = run_query('SELECT * FROM park LIMIT 5')
result

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [10]:
q = '''
INSERT OR IGNORE INTO league VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "Double-A Minor League"),
    ("UA", "Union Association League"),
    ("FL", "Florida State League"),
    ("PL", "Players' League")
'''
run_command(q)

result = run_query('SELECT * FROM league')
result
# teamc['league'].value_counts()

,league_id,name
0,NL,National League
1,AL,American League
2,AA,Double-A Minor League
3,UA,Union Association League
4,FL,Florida State League
5,PL,Players' League


In [11]:
q = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(q)

q = '''
INSERT OR IGNORE INTO team 
    SELECT                                   
       team_id,                                                           
       league,
       city,
       nickname,
       franch_id
    FROM team_codes
'''
run_command(q)

result = run_query('SELECT * FROM team')
result.head()

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [12]:
q = '''
CREATE TABLE IF NOT EXISTS game(
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day INTEGER,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info INTEGER,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
'''
run_command(q)

q = '''
INSERT OR IGNORE INTO game 
    SELECT                                   
        game_id,
        date,
        number_of_game,
        park_id TEXT,
        length_outs,
        case when day_night="D" then 1 else 0 end as day,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        case when acquisition_info="Y" then 1 else 0 end as day
    FROM game_log
'''
run_command(q)

result = run_query('SELECT * FROM game LIMIT 5')
result

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,1
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,1
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,1
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,1
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,1


In [13]:
q = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home TEXT,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
'''
run_command(q)

q = '''
INSERT OR IGNORE INTO team_appearance 
    SELECT                                   
        h_name,
        game_id,
        1,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays        
    FROM game_log
    
    UNION
    
    SELECT                                   
        v_name,
        game_id,
        0,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays        
    FROM game_log
'''
run_command(q)

result = run_query('SELECT * FROM team_appearance LIMIT 5')
result

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [68]:
run_command('DROP TABLE IF EXISTS person_appearance')
run_command('DROP TABLE IF EXISTS appearance_type')

q = '''
CREATE TABLE IF NOT EXISTS appearance_type(
    appearance_type_id TEXT,
    name TEXT,
    category TEXT,
    PRIMARY KEY (appearance_type_id))
'''
run_command(q)

with sqlite3.connect('mlb.db') as conn:
    q='''INSERT OR REPLACE INTO appearance_type 
    (appearance_type_id, name, category) VALUES (?,?,?)'''
    conn.executemany(q, appeart.to_records(index=False))
    conn.commit()

# appeart.to_sql('appearance_type', conn, index_label='appearance_type_id')

q = '''
CREATE TABLE IF NOT EXISTS person_appearance(
    appearence_id INTEGER,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    PRIMARY KEY (appearence_id),
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
)
;
'''
run_command(q)

q = ''' 
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        NULL,
        [lf_umpire_id],
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL
'''
run_command(q)

result = run_query('SELECT * FROM person_appearance LIMIT 5')
result

,appearence_id,person_id,team_id,game_id,appearance_type_id
0,1,curte801,ALT,ALT188404300,MM
1,2,murpj104,ALT,ALT188404300,PSP
2,3,hodnc101,SLU,ALT188404300,PSP
3,4,sullt101,SLU,ALT188404300,MM
4,5,curte801,ALT,ALT188405020,MM


In [70]:
template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template.format(**query_vars))
        
result = run_query('SELECT * FROM person_appearance LIMIT 5')
result

,appearence_id,person_id,team_id,game_id,appearance_type_id
0,1,curte801,ALT,ALT188404300,MM
1,2,murpj104,ALT,ALT188404300,PSP
2,3,hodnc101,SLU,ALT188404300,PSP
3,4,sullt101,SLU,ALT188404300,MM
4,5,curte801,ALT,ALT188405020,MM


In [71]:
run_command('DROP TABLE IF EXISTS game_log')
run_command('DROP TABLE IF EXISTS park_codes')
run_command('DROP TABLE IF EXISTS team_codes')
run_command('DROP TABLE IF EXISTS person_codes')

bonus:
Transform the the dates into a SQLite compatible format.
Extract the line scores into innings level data in a new table.
Create views to make querying stats easier, eg:
Season level stats.
All time records.
Supplement the database using new data, for instance:
Add data from retrosheet game logs for years after 2016.
Source and add missing pitcher information.
Add player level per-game stats.
Source and include base coach data.